# Setup

1. Connect Gdrive !

In [2]:
!cp /content/drive/MyDrive/encoder_stylegan2ada/best-encoder-parameters.pt .

In [3]:
!cp /content/drive/MyDrive/STYLEGAN_LATENT_RESULTS/latent_directions/age.npy .

In [4]:
# Git Clone
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 2.98 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [5]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.3 MB/s eta 0:04:16tcmalloc: large alloc 1147494400 bytes == 0x39a0e000 @  0x7f5796c08615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:28tcmalloc: large alloc 1434370048 bytes == 0x7e064000 @  0x7f5796c08615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [6]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 8.1 MB/s 
     |████████████████████████████████| 108 kB 69.7 MB/s 
     |████████████████████████████████| 26.9 MB 1.2 MB/s 


In [7]:
!gdown https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Downloading...
From: https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
To: /content/ffhq.pkl
100% 382M/382M [00:01<00:00, 278MB/s]


In [8]:
import sys
sys.path.insert(0, "/content/stylegan2-ada-pytorch")

In [9]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
import torch
import pickle

with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [11]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/shape_predictor_68_face_landmarks.dat.bz2

--2022-06-15 08:01:47--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  27.5MB/s    in 2.2s    

2022-06-15 08:01:49 (27.5 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [12]:
"""
brief: face alignment with FFHQ method (https://github.com/NVlabs/ffhq-dataset)
author: lzhbrian (https://lzhbrian.me)
date: 2020.1.5
note: code is heavily borrowed from 
    https://github.com/NVlabs/ffhq-dataset
    http://dlib.net/face_landmark_detection.py.html

requirements:
    apt install cmake
    conda install Pillow numpy scipy
    pip install dlib
    # download face landmark model from: 
    # http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
"""

import numpy as np
import PIL
import PIL.Image
import sys
import os
import glob
import scipy
import scipy.ndimage
import dlib


# download model from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

def get_landmark(filepath):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)

    print("Number of faces detected: {}".format(len(dets)))
    for k, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k, d.left(), d.top(), d.right(), d.bottom()))
        # Get the landmarks/parts for the face in box d.
        shape = predictor(img, d)
        print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))


    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    # lm is a shape=(68,2) np.array
    return lm

    
def align_face(filepath):
    """
    :param filepath: str
    :return: PIL Image
    """

    lm = get_landmark(filepath)
    
    lm_chin          = lm[0  : 17]  # left-right
    lm_eyebrow_left  = lm[17 : 22]  # left-right
    lm_eyebrow_right = lm[22 : 27]  # left-right
    lm_nose          = lm[27 : 31]  # top-down
    lm_nostrils      = lm[31 : 36]  # top-down
    lm_eye_left      = lm[36 : 42]  # left-clockwise
    lm_eye_right     = lm[42 : 48]  # left-clockwise
    lm_mouth_outer   = lm[48 : 60]  # left-clockwise
    lm_mouth_inner   = lm[60 : 68]  # left-clockwise

    # Calculate auxiliary vectors.
    eye_left     = np.mean(lm_eye_left, axis=0)
    eye_right    = np.mean(lm_eye_right, axis=0)
    eye_avg      = (eye_left + eye_right) * 0.5
    eye_to_eye   = eye_right - eye_left
    mouth_left   = lm_mouth_outer[0]
    mouth_right  = lm_mouth_outer[6]
    mouth_avg    = (mouth_left + mouth_right) * 0.5
    eye_to_mouth = mouth_avg - eye_avg

    # Choose oriented crop rectangle.
    x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
    x /= np.hypot(*x)
    x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
    y = np.flipud(x) * [-1, 1]
    c = eye_avg + eye_to_mouth * 0.1
    quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
    qsize = np.hypot(*x) * 2


    # read image
    img = PIL.Image.open(filepath)

    output_size=1024
    transform_size=4096
    enable_padding=True

    # Shrink.
    shrink = int(np.floor(qsize / output_size * 0.5))
    if shrink > 1:
        rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
        img = img.resize(rsize, PIL.Image.ANTIALIAS)
        quad /= shrink
        qsize /= shrink

    # Crop.
    border = max(int(np.rint(qsize * 0.1)), 3)
    crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
    if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
        img = img.crop(crop)
        quad -= crop[0:2]

    # Pad.
    pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
    if enable_padding and max(pad) > border - 4:
        pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
        img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
        h, w, _ = img.shape
        y, x, _ = np.ogrid[:h, :w, :1]
        mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
        blur = qsize * 0.02
        img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
        img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
        img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
        quad += pad[:2]

    # Transform.
    img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
    if output_size < transform_size:
        img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

    # Save aligned image.
    return img




In [13]:
import torch.nn as nn
from torchvision import models as models
import torch.nn.functional as F

class StyleganEncoder(nn.Module):
    def __init__(self):
        super(StyleganEncoder, self).__init__()
        self.feature_extractor = self._getResnet()
        self.conv_1 = nn.Conv2d(512, 256, 1)
        self.conv_2 = nn.Conv2d(256, 144, 1)
        self.mapping_network = nn.Sequential(
          nn.Linear(96,96),
          nn.ReLU(),
          nn.Linear(96,96),
          nn.ReLU()
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        batch_dim = x.shape[0]
        x = torch.reshape(x, (batch_dim, 96, 96))
        x = self.mapping_network(x)
        x = torch.reshape(x, (batch_dim, 18, 512))
        return x

    def _getResnet(self):
        resnet = models.resnet18(pretrained=True)
        features = nn.Sequential(*(list(resnet.children())[0:8]))
        for param in features.parameters():
            param.requires_grad = False
        # uciac do basic block 3-8
        # dodac conv2d  output 8|8| 144
        # reshape 96x96
        # mapping network 96x96
        # reshape 18,512
        return features

resnet = StyleganEncoder()
x = torch.randn(10, 3, 256, 256)
output = resnet(x)
output.shape

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

torch.Size([10, 18, 512])

In [14]:
import PIL
import numpy as np
import torchvision.transforms as transforms

def preprocess_image(img_path):
    img = PIL.Image.open(img_path)
    img = TRANSFORM_IMG(img)
    img = img.unsqueeze(0).to(device)
    return img

def load_encoder(weights_pth):
    encoder = StyleganEncoder().to(device)
    encoder.load_state_dict(torch.load(weights_pth, map_location=device))
    return encoder

def get_encoder_latent(img, encoder, save=False):
    latent_img = encoder(img)
    if save is True:
        img_embedded = G.synthesis(latent_img, noise_mode='const', force_fp32=True)
        img_embedded = (img_embedded.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img_embedded[0].cpu().numpy(), 'RGB').save(f'encoder_img.png')
    return latent_img


In [15]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import PIL


def read_img(image_path):
    '''
    1. Read IMG
    2. Preprocess:
     - cut (eyes are points)
     - resize
    3. Transform to Torch
    '''
    img = PIL.Image.open(image_path)
    transform = transforms.ToTensor() # normalization 0 to 1
    img = transform(img)
    img = torch.unsqueeze(img, 0)
    upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
    
    return img

def perceptual_loss(model, image_mean, image_embed):
    image_mean = image_mean.to(device)
    image_embed = image_embed.to(device)
    features_mean = model(image_mean)
    features_embed = model(image_embed)
    ppl_loss = 0
    for i in range(len(features_mean)):
        ppl_loss += mse_loss(features_mean[i],features_embed[i])
    return ppl_loss

def mse_loss(image_mean, image_embed):
    loss = nn.MSELoss()
    image_mean = image_mean.to(device)
    image_embed = image_embed.to(device)
    output = loss(image_mean, image_embed)
    return output

def loss_all(model, image_mean, image_embed, upsample):
    image_mean_up = upsample(image_mean)
    image_embed_up = upsample(image_embed)
    ppl = perceptual_loss(model, image_mean_up, image_embed_up) # tutaj upsample
    mse = mse_loss(image_mean, image_embed)
    return ppl, mse

In [16]:
from torchvision import models

class VGG16_perceptual(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(VGG16_perceptual, self).__init__()
        vgg_pretrained_features = models.vgg16(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 4):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 14):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(14, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_1 = h
        h = self.slice2(h)
        h_relu1_2 = h
        h = self.slice3(h)
        h_relu3_2 = h
        h = self.slice4(h)
        h_relu4_2 = h
        return h_relu1_1, h_relu1_2, h_relu3_2, h_relu4_2

In [17]:
def get_embedding(model, G, optimizer, latent_encoder, image_to_embed, upsample, step=5000):
    '''
    dopisac .to(device)
    '''
    loss_ = []
    for i in range(step):
        optimizer.zero_grad()
        image_mean = G.synthesis(latent_encoder, noise_mode='const', force_fp32=True)
        image_mean = (image_mean+1.0)/2.0 # from union -1|1 to 0/1
        per_loss, mse = loss_all(model, image_mean, image_to_embed, upsample)
        loss = per_loss + mse
        loss.backward()
        optimizer.step()
        loss_.append(loss.detach().cpu().numpy())
    return loss_, latent_encoder

In [28]:
weights_pth = "/content/best-encoder-parameters.pt"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

def get_child_image(father_img="/content/father.jpg", mother_img="/content/mother.jpg", feature_vector="age.npy"):
    '''
    feature_vector SHAPE: 1,18,512 Young/Old direction
    '''

    def save_image_from_latent(latent, img_path):
        img_to_save = G.synthesis(latent, noise_mode='const', force_fp32=True)
        img_to_save = (img_to_save.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img_to_save[0].cpu().numpy(), 'RGB').save(f'{img_path}')

    feature_vector = np.load(feature_vector)
    feature_vector = torch.from_numpy(feature_vector)
    feature_vector = feature_vector.to(device)
    img_f =  align_face(father_img)
    img_f = img_f.save("father_aligned.png")
    img_m =  align_face(mother_img) 
    img_m = img_m.save("mother_aligned.png")
    ############

    encoder = load_encoder(weights_pth)

    father_to_encoder = preprocess_image("father_aligned.png")
    father_latent_encoder = get_encoder_latent(father_to_encoder, encoder, save=False)
    mother_to_encoder = preprocess_image("mother_aligned.png")
    mother_latent_encoder = get_encoder_latent(mother_to_encoder, encoder, save=True)
    #############

    upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
    model = VGG16_perceptual().to(device)

    father_latent_encoder = torch.tensor(father_latent_encoder, requires_grad=True)
    optim_father = torch.optim.Adam({father_latent_encoder},lr=0.01,betas=(0.9,0.999),eps=1e-8)
    mother_latent_encoder = torch.tensor(mother_latent_encoder, requires_grad=True)
    optim_mother = torch.optim.Adam({mother_latent_encoder},lr=0.01,betas=(0.9,0.999),eps=1e-8)
    #############

    father_to_embed = read_img("father_aligned.png")
    loss_, father_final_latent = get_embedding(model, G, optim_father, father_latent_encoder, father_to_embed, upsample, step=300)
    mother_to_embed = read_img("mother_aligned.png")
    loss_, mother_final_latent = get_embedding(model, G, optim_mother, mother_latent_encoder, mother_to_embed, upsample, step=300)
    save_image_from_latent(father_final_latent, "father_final_embedding.jpg")
    save_image_from_latent(mother_final_latent, "mother_final_embedding.jpg")

    ########### Calculate child latent
    child_latent = torch.add(father_final_latent, mother_final_latent)
    child_latent = child_latent/2 # moze sie wyhebac tu
    factor = 2.5 # CHECK AND CHANGE
    feature_vector = (feature_vector*factor)
    child_latent = torch.add(child_latent, feature_vector)
    ########### Save
    img_child = G.synthesis(child_latent, noise_mode='const', force_fp32=True)
    img_child = (img_child.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img_child[0].cpu().numpy(), 'RGB').save(f'Child.png')
    return child_latent



In [29]:
child_latent = get_child_image()

Number of faces detected: 1
Detection 0: Left: 118 Top: 333 Right: 504 Bottom: 718
Part 0: (98, 457), Part 1: (104, 507) ...
Number of faces detected: 1
Detection 0: Left: 171 Top: 295 Right: 726 Bottom: 850
Part 0: (256, 504), Part 1: (262, 576) ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/u

In [ ]:
#child_latent
img_child = G.synthesis(child_latent, noise_mode='const', force_fp32=True)
img_child = (img_child.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img_child[0].cpu().numpy(), 'RGB').save(f'Child.png')